In [3]:
from author_style.utils import *
import tensorflow

In [13]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import unidecode
#import spacy
import numpy as np
import pandas as pd

In [22]:
df = csv_to_dataframes(output="s")

In [23]:
df.head()

,text,author,title,book_date
0,Pendant que Nous franchissions la porte du Nor...,GUTH Paul,Si j_étais le Bon Dieu,1987
1,Pour compenser la fatigue que pourrait causer ...,GUTH Paul,Si j_étais le Bon Dieu,1987
2,"Un silence de rêve suspendrait, sur la terre, ...",GUTH Paul,Si j_étais le Bon Dieu,1987
3,"Il éteindrait, dans l’univers, la musique des ...",GUTH Paul,Si j_étais le Bon Dieu,1987
4,Cette semaine mensuelle de silence serait voué...,GUTH Paul,Si j_étais le Bon Dieu,1987


In [24]:
data = df[["text","author"]]

In [25]:
def preprocess(text,
               punctuation=True,
               lower_case=True,
               remove_stopwords=True,
               accents=True,
               numbers=True,
               lemmatize=False,
               language='french'):

    if numbers:
        text = ''.join(char for char in text if not char.isdigit())
    if punctuation:
        text = ''.join(char for char in text if not char in string.punctuation)
        text = text.replace("*","")
    if lower_case:
        text = text.lower()
    if accents:
        text = unidecode.unidecode(text)
    if remove_stopwords:
        stop_words = set(stopwords.words(language))
        word_tokens = word_tokenize(text)
        text = ' '.join(char for char in word_tokens if not char in stop_words)
    if lemmatize:
        text = word_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        lemmatized = [lemmatizer.lemmatize(char) for char in text]
        text = ' '.join(lemmatized)
    return text

In [26]:
data["text"] = data["text"].apply(preprocess)

/var/folders/t8/clhcfp2n5sgcs4b81syf8xym0000gp/T/ipykernel_16919/1989513621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(preprocess)


In [27]:
data.shape

(284865, 2)

In [28]:
data["taille"] = data.text.apply(lambda x : len(x))

/var/folders/t8/clhcfp2n5sgcs4b81syf8xym0000gp/T/ipykernel_16919/92849788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["taille"] = data.text.apply(lambda x : len(x))


In [29]:
data

,text,author,taille
0,pendant franchissions porte nord dediee a isht...,GUTH Paul,137
1,compenser fatigue pourrait causer jacassement ...,GUTH Paul,110
2,silence reve suspendrait terre grondement mers...,GUTH Paul,121
3,eteindrait l'univers musique spheres tout frem...,GUTH Paul,58
4,cette semaine mensuelle silence vouee a l'amour,GUTH Paul,47
...,...,...,...
284860,lendemain matin leve tard trainant lit jusqu '...,ECHENOZ Jean,249
284861,part direction gare maritime havre afin rendre...,ECHENOZ Jean,60
284862,c'est premiere fois qu'il va derniere,ECHENOZ Jean,37
284863,reste aujourd'hui pile dix ans a vivre,ECHENOZ Jean,38


In [32]:
X_sen = data["text"].to_numpy()
y_sen = data["author"].to_numpy()

In [33]:
import joblib

joblib.dump(X_sen,"X_sen.pkl")
joblib.dump(y_sen,"y_sen.pkl")

['y_sen.pkl']